In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from component import tile
from component import model
from sepal_ui.planetapi import PlanetView

In [ ]:
tb_model = model.TableModel()
# tiles to display the points on a map
tb_map_tile = tile.MapTile()
# output to display messages
file_tile = tile.FileTile(tb_model, tb_map_tile.map)
# create a test downloader
test_tile = tile.TestTile(file_tile)

In [ ]:
planet_view = PlanetView()

In [ ]:
viz_model = model.VizModel()
viz_input_tile = tile.InputTile(viz_model, tb_model, planet_view=planet_view)

In [ ]:
export_model = model.ExportModel()
# result tile
export_results = tile.ExportResult()
# export data
export_tile = tile.ExportData(
    export_model,
    viz_model,
    tb_model,
    export_results,
    planet_model=planet_view.planet_model,
)

In [ ]:
test_tile

In [ ]:
file_tile

In [ ]:
tb_map_tile

In [ ]:
viz_input_tile

In [ ]:
export_tile

In [ ]:
from component.scripts.gee import gee_ee_image

In [ ]:
import threading
import concurrent.futures

from typing import Literal, Tuple
import zipfile
from functools import partial
from pathlib import Path
from urllib.request import urlretrieve

import ee
from osgeo import gdal
from sepal_ui import sepalwidgets as sw
from sepal_ui.scripts.utils import init_ee

from component import parameter as cp
from component import widget as cw
from component.message import cm

from component.scripts.utils import get_buffers, get_vrt_filename, min_diagonal

In [ ]:
def get_ee_image(
    satellites: dict,
    satellite_id: Literal["sentinel_2", "landsat_5", "landsat_7", "landsat_8"],
    start: str,
    end: str,
    str_bands: str,
    aoi: ee.geometry.Geometry,
) -> Tuple[ee.ImageCollection, ee.Image]:

    # create the feature collection name
    dataset = (
        ee.ImageCollection(satellites[satellite_id])
        .filterDate(start, end)
        .filterBounds(aoi)
        .map(cp.getCloudMask(satellite_id))
    )

    bands = cp.getAvailableBands()[str_bands][satellite_id]
    ee_image = dataset.median().clip(aoi).select(bands)

    # calculate the NDVI or NDWI if needed
    # Bands are in the correct order to do the index calculation
    if "ndvi" in str_bands or "ndwi" in str_bands:
        ee_image = ee_image.normalizedDifference(bands).rename("ndvi")

    return dataset, ee_image

In [ ]:
sources = ["sentinel", "landsat"]

aoi = ee.Geometry.Polygon(
    [
        [
            [13.024513100356552, 5.333572819469696],
            [13.026757769061255, 5.333572819469696],
            [13.026757769061255, 5.335822103684232],
            [13.024513100356552, 5.335822103684232],
            [13.024513100356552, 5.333572819469696],
        ]
    ]
)

# Test with Sentinel_2
bands = str_bands = "Nir, Swir1, Red"
satellite_id = "landsat_5"
year = 1990
start = str(year) + "-01-01"
end = str(year) + "-12-31"
satellites = cp.getSatellites(sources, year)
dataset, ee_image = get_ee_image(satellites, satellite_id, start, end, bands, aoi)

In [ ]:
# create the feature collection name
dataset = (
    ee.ImageCollection(satellites[satellite_id])
    .filterDate(start, end)
    .filterBounds(aoi)
    .map(cp.getCloudMask(satellite_id))
)
bands = cp.getAvailableBands()[str_bands][satellite_id]
ee_image = dataset.median().clip(aoi)

In [ ]:
aoi

In [ ]:
satellites[satellite_id]